In [1]:
# coding:utf-8

# keras公式：https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py を使用

from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
epochs = 130  # Number of epochs to train for.

batch_size = 64  # Batch size for training.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Path to the data txt file on disk.
data_path = './dataset.txt'

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
# dataset
# mk_dataset.ipynbで作成
with open("./dataset.txt", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [5]:
lines[0]

'フシギダネ\tフシギソウ'

In [6]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 330
Number of unique input tokens: 74
Number of unique output tokens: 79
Max sequence length for inputs: 6
Max sequence length for outputs: 9


In [7]:
print(input_characters)

['ア', 'ィ', 'イ', 'ウ', 'ェ', 'エ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'ゲ', 'コ', 'ゴ', 'サ', 'ザ', 'シ', 'ジ', 'ス', 'ズ', 'セ', 'ゼ', 'ソ', 'ゾ', 'タ', 'ダ', 'チ', 'ッ', 'ツ', 'テ', 'デ', 'ト', 'ド', 'ナ', 'ニ', 'ヌ', 'ネ', 'ノ', 'ハ', 'バ', 'パ', 'ヒ', 'ビ', 'ピ', 'フ', 'ブ', 'プ', 'ヘ', 'ベ', 'ホ', 'ボ', 'ポ', 'マ', 'ミ', 'ム', 'メ', 'モ', 'ャ', 'ヤ', 'ュ', 'ユ', 'ョ', 'ヨ', 'ラ', 'リ', 'ル', 'レ', 'ロ', 'ワ', 'ン', 'ー']


In [8]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [9]:
encoder_input_data.shape

(330, 6, 74)

In [10]:
decoder_input_data.shape

(330, 9, 79)

In [11]:
decoder_target_data.shape

(330, 9, 79)

In [12]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 74)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 79)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 338944      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  344064      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [13]:
#model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
# Save model
#model.save('s2s.h5')

In [14]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs)

Epoch 1/130
330/330 [==============================] - 2s 5ms/step - loss: 2.6925
Epoch 2/130
330/330 [==============================] - 0s 1ms/step - loss: 2.2713
Epoch 3/130
330/330 [==============================] - 0s 1ms/step - loss: 2.2031
Epoch 4/130
330/330 [==============================] - 0s 2ms/step - loss: 2.2137
Epoch 5/130
330/330 [==============================] - 0s 1ms/step - loss: 2.1581
Epoch 6/130
330/330 [==============================] - 0s 1ms/step - loss: 2.2031
Epoch 7/130
330/330 [==============================] - 0s 1ms/step - loss: 2.1516
Epoch 8/130
330/330 [==============================] - 0s 1ms/step - loss: 2.1512
Epoch 9/130
330/330 [==============================] - 0s 1ms/step - loss: 2.1360
Epoch 10/130
330/330 [==============================] - 0s 1ms/step - loss: 2.1132
Epoch 11/130
330/330 [==============================] - 0s 1ms/step - loss: 2.1094
Epoch 12/130
330/330 [==============================] - 0s 1ms/step - loss: 2.1026
Epoch 13/130


330/330 [==============================] - 1s 2ms/step - loss: 0.5539
Epoch 100/130
330/330 [==============================] - 1s 2ms/step - loss: 0.6044
Epoch 101/130
330/330 [==============================] - 1s 2ms/step - loss: 0.5609
Epoch 102/130
330/330 [==============================] - 1s 2ms/step - loss: 0.5121
Epoch 103/130
330/330 [==============================] - 1s 2ms/step - loss: 0.4919
Epoch 104/130
330/330 [==============================] - 1s 2ms/step - loss: 0.4881
Epoch 105/130
330/330 [==============================] - 0s 1ms/step - loss: 0.5721
Epoch 106/130
330/330 [==============================] - 1s 2ms/step - loss: 0.4756
Epoch 107/130
330/330 [==============================] - 1s 2ms/step - loss: 0.4329
Epoch 108/130
330/330 [==============================] - 1s 2ms/step - loss: 0.4081
Epoch 109/130
330/330 [==============================] - 1s 2ms/step - loss: 0.4412
Epoch 110/130
330/330 [==============================] - 0s 1ms/step - loss: 0.4035
Epoch 

In [15]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [16]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [17]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: フシギダネ
Decoded sentence: フシギソウ

-
Input sentence: フシギソウ
Decoded sentence: フシギバナ

-
Input sentence: ヒトカゲ
Decoded sentence: リザード

-
Input sentence: リザード
Decoded sentence: リザードン

-
Input sentence: ゼニガメ
Decoded sentence: カメール

-
Input sentence: カメール
Decoded sentence: カメックス

-
Input sentence: キャタピー
Decoded sentence: トランセル

-
Input sentence: トランセル
Decoded sentence: バタフリー

-
Input sentence: ビードル
Decoded sentence: コクーン

-
Input sentence: コクーン
Decoded sentence: スピアー



In [18]:
def test_model(input_sentence):
    test_sentence_tokenized = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for t, char in enumerate(input_sentence):
        test_sentence_tokenized[0, t, input_token_index[char]] = 1.
    #print(input_sentence)
    print(decode_sequence(test_sentence_tokenized))

In [19]:
test_model("リスプ")

スリーパ



In [20]:
test_model("パイソン")

シンテー



In [21]:
test_model("ヒトシ")

トリプー



In [22]:
test_model("フシギ")

フシギバナ



In [23]:
test_model("フシ")

フイバー



In [24]:
test_model("シギ")

ギギアル

